In [1]:
# Import Tools

import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [2]:
tag_urls = {
    'Love': 'https://medium.com/tag/lov/archive/{0}/{1:02d}/{2:02d}'
}

In [3]:
year = 2022
selected_days = [i for i in range(1, 48)] #Every day of the year

In [4]:
def convert_day(day):
    # if it is a leap year use month_days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [5]:
def get_claps(claps_str):
    if (claps_str is None) or (claps_str == '') or (claps_str.split is None):
        return 0
    split = claps_str.split('K')
    claps = float(split[0])
    claps = int(claps*1000) if len(split) == 2 else int(claps)
    return claps

In [6]:
articles_data = []
article_id = 0
n = len(selected_days)
for d in selected_days:
    month, day = convert_day(d)
    date = '{0}-{1:02d}-{2:02d}'.format(year, month, day) 
    print(f'{date}')
    for tag, url in tag_urls.items(): 
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all(
            "div",
            class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        
        for article in articles:
            
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            title = title.contents[0]
            
            author = article.find_all("a")[0]['href'].split('?')[0].split('@')[1]
            author_url = article.find_all("a")[0]['href'].split('?')[0]
            
            subtitle = article.find("h4", class_="graf--subtitle")
            subtitle = subtitle.contents[0] if subtitle is not None else ''
            
            article_url = article.find_all("a")[3]['href'].split('?')[0]
            
            reading_time = article.find("span", class_="readingTime")
            reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
            
            responses = article.find_all("a")
            if len(responses) == 7:
                responses = responses[6].contents[0].split(' ')
                if len(responses) == 0:
                    responses = 0
                else:
                    responses = responses[0]
            else:
                responses = 0
                
            articles_data.append([article_url, title,
                         author, author_url,
                         subtitle, responses,
                         reading_time, tag, date])

2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16


In [7]:
# Transform article data into panda dataframe.
medium_df = pd.DataFrame(articles_data, columns=[
    'url', 'title', 'author', 'author_page',
    'subtitle', 'responses', 'reading_time',
    'tag', 'date'])

In [8]:
medium_df.shape

(8906, 9)

In [9]:
medium_df = medium_df.drop_duplicates(subset=['url', 'title'], keep='first')

In [10]:
medium_df.shape

(8906, 9)

In [11]:
medium_df

,url,title,author,author_page,subtitle,responses,reading_time,tag,date
0,https://medium.com/hello-love/7-out-of-10-rela...,[7 Out of 10 Relationships Went Into Crisis. T...,goodmenproject,https://medium.com/@goodmenproject,[No one had an affair. It wasn’t about money],4,7,Love,2022-01-01
1,https://medium.com/@thoughtcatalog/7-things-yo...,7 Things Your Relationship Should Be Able To W...,thoughtcatalog,https://medium.com/@thoughtcatalog,You probably aren’t the first person your…,0,3,Love,2022-01-01
2,https://medium.com/on-the-couch/10-popular-cli...,10 Popular Clichés About Love You Should Absol...,karennimmo,https://medium.com/@karennimmo,Be careful of what your mother told you.,8,5,Love,2022-01-01
3,https://medium.com/@sustainablesong/how-ill-di...,"How I’ll Die, When and Why",sustainablesong,https://medium.com/@sustainablesong,,0,7,Love,2022-01-01
4,https://medium.com/bella-cooper-books/becoming...,"Becoming Best Friends, With My Husband’s BFF",bellacooperbooks,https://medium.com/@bellacooperbooks,I fucked up and sent a…,1,8,Love,2022-01-01
...,...,...,...,...,...,...,...,...,...
8901,https://medium.com/poeticessences/unintended-8...,Unintended,un-ptit-je-ne-sais-quoi,https://medium.com/@un-ptit-je-ne-sais-quoi,MicroPoem,2,1,Love,2022-02-16
8902,https://medium.com/@writersyep/conscious-creat...,Conscious Creatives: Collective General Readin...,writersyep,https://medium.com/@writersyep,"Healing with joy, grace…",0,22,Love,2022-02-16
8903,https://medium.com/@d-davidcroot/if-transcende...,If transcendence were to be achieved,d-davidcroot,https://medium.com/@d-davidcroot,,0,1,Love,2022-02-16
8904,https://medium.com/@kmollion/falling-in-love-3...,Falling in Love,kmollion,https://medium.com/@kmollion,This is what it feels like for those who don’t...,0,1,Love,2022-02-16


In [12]:
medium_df.to_csv('medium-data-love.csv', index=True)